In [2]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay")

okay


In [3]:
import pickle

rsfile = open('/Users/barrybrent/13dec22no1.txt','rb')
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)
print(len(s))

rtfile = open('/Users/barrybrent/13dec22no2.txt','rb') # see cell below
rt=pickle.load(rtfile)
rtfile.close()
t=stripQuotationMarks(rt)
print(len(t))

u=[]
for j in s:u=u+[j]
for j in t:u=u+[j]
print(len(u)) 
# this takes so long to make that I will
# reserve the letter y for the file created on lines 15-17.
# in all cells below. I won't reload, as I usually do,
# in every cell.

100
20
120


In [ ]:
#output snipped
polydata=[]

import time
for power in [1..100]:
    start=time.time()/3600
    data=[]
    for m in [3..3*power+3]:
        cst= JpowerConstantTerm(m,power)
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    deg=rl.degree()
    polydata=polydata+[[power,rl]]
    finish=time.time()/3600
    print("(power,degree,time): ",(power,deg,finish-start))

import pickle
wfile = open('/Users/barrybrent/13dec22no1.txt','wb')
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()
############################################################################################################
polydata=[]

import time
for power in [101..120]:
    start=time.time()/3600
    data=[]
    for m in [3..3*power+3]:
        cst= JpowerConstantTerm(m,power)
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    deg=rl.degree()
    polydata=polydata+[[power,rl]]
    finish=time.time()/3600
    print("(power,degree,time): ",(power,deg,finish-start))

import pickle
wfile = open('/Users/barrybrent/13dec22no2.txt','wb')
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

In [4]:
for k in [1..5]:
    print((k,broadhurst(k)))

(1, 1)
(2, 47)
(3, 2488)
(4, 138799)
(5, 7976456)


In [5]:
order(2,3/4)

-2

In [6]:
print(order(2,1/205915392))

-8


In [3]:
data=[]
for n in [2..99]:
    nsz2=order(2,ZZ(broadhurst(n)))
    ds2=digitsum(2,n)
    data=data+[nsz2-(3*ds2-3)]
print(data)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
residues=[]
for k in [0..len(u)-1]:
    data=[]
    power=u[k][0]
    poly=D(u[k][1]) # poly(m) = cst term of j_m^power
    nf=QQ(numericalfactor(poly))
    polyd=expand(poly/nf)
    v=evaluate(polyd,3)
    residue=mod(v,2)
    residues=residues+[residue]
print(residues) # so, within the range of observations,ord(2,cst term of j_3^power) = ord(2,nf)
# i.e., ord(2,cst term of j^power) = ord(2,nf) and per cell above, ord(2,nf)
# obeys the 2-order rule in my '98 Exp. Math. article.

In [17]:
for j in [0..5]:
    print("----------------------------------------------------------------------------")
    power=u[j][0]
    print("power:",power)
    poly=D(u[j][1]) # poly(m) = cst term of j_m^power
    #nf=QQ(numericalfactor(poly))
    #polyd=expand(poly/nf)
    print(polyd.factor())
    v=evaluate(poly,3)
    d2=digitsum(2,power)
    d3=digitsum(3,power)
    o2=order(2,v)
    o3=order(3,v)
    print((d2,-o2,d3,-o3))
    
    
 

----------------------------------------------------------------------------
power: 1
x^6 - 2409555566418/58638404089*x^5 + 41294152300198/58638404089*x^4 - 380038442003220/58638404089*x^3 + 2002024211631241/58638404089*x^2 - 5843938015317402/58638404089*x + 12868026480268632/58638404089
(1, 3, 1, 2)
----------------------------------------------------------------------------
power: 2
x^6 - 2409555566418/58638404089*x^5 + 41294152300198/58638404089*x^4 - 380038442003220/58638404089*x^3 + 2002024211631241/58638404089*x^2 - 5843938015317402/58638404089*x + 12868026480268632/58638404089
(1, 9, 2, 4)
----------------------------------------------------------------------------
power: 3
x^6 - 2409555566418/58638404089*x^5 + 41294152300198/58638404089*x^4 - 380038442003220/58638404089*x^3 + 2002024211631241/58638404089*x^2 - 5843938015317402/58638404089*x + 12868026480268632/58638404089
(2, 12, 1, 8)
----------------------------------------------------------------------------
power: 4
x^6 - 2

In [18]:
for j in [0..5]:
    power=u[j][0]
    #print("power:",power)
    poly=D(u[j][1]) # poly(m) = cst term of j_m^power
    #nf=QQ(numericalfactor(poly))
    #polyd=expand(poly/nf)
    #print(polyd.factor())
    v=evaluate(poly,3)
    d2=digitsum(2,power)
    d3=digitsum(3,power)
    o2=order(2,v)
    o3=order(3,v)
    print((power,d2,-o2,d3,-o3))

(1, 1, 3, 1, 2)
(2, 1, 9, 2, 4)
(3, 2, 12, 1, 8)
(4, 1, 21, 2, 10)
(5, 2, 24, 3, 12)
(6, 2, 30, 2, 16)


In [24]:
data2=[];data3=[]
for j in [0..20]:
    power=u[j][0]
    #print("power:",power)
    poly=D(u[j][1]) # poly(m) = cst term of j_m^power
    #nf=QQ(numericalfactor(poly))
    #polyd=expand(poly/nf)
    #print(polyd.factor())
    v=evaluate(poly,3)
    d2=digitsum(2,power)
    d3=digitsum(3,power)
    o2=order(2,v)
    o3=order(3,v)
    data2=data2+[-o2];data3=data3+[-o3]
print(data2);print()
dt2=difference_table(data2)
print(dt2);print()
print(data3);print()
dt3=difference_table(data3)
print(dt3)

[3, 9, 12, 21, 24, 30, 33, 45, 48, 54, 57, 66, 69, 75, 78, 93, 96, 102, 105, 114, 117]

[6, 3, 9, 3, 6, 3, 12, 3, 6, 3, 9, 3, 6, 3, 15, 3, 6, 3, 9, 3]

[2, 4, 8, 10, 12, 16, 18, 20, 26, 28, 30, 34, 36, 38, 42, 44, 46, 52, 54, 56, 60]

[2, 4, 2, 2, 4, 2, 2, 6, 2, 2, 4, 2, 2, 4, 2, 2, 6, 2, 2, 4]


In [26]:
data2=[];data3=[]
for j in [0..20]:
    power=u[j][0]
    #print("power:",power)
    poly=D(u[j][1]) # poly(m) = cst term of j_m^power
    #nf=QQ(numericalfactor(poly))
    #polyd=expand(poly/nf)
    #print(polyd.factor())
    v=evaluate(poly,3)
    d2=digitsum(2,power)
    d3=digitsum(3,power)
    o2=order(2,v)
    o3=order(3,v)
    data2=data2+[-o2/d2];data3=data3+[-o3/d3]
print(data2);print()
dt2=difference_table(data2)
print(dt2);print()
print(data3);print()
dt3=difference_table(data3)
print(dt3)

[3, 9, 6, 21, 12, 15, 11, 45, 24, 27, 19, 33, 23, 25, 39/2, 93, 48, 51, 35, 57, 39]

[6, -3, 15, -9, 3, -4, 34, -21, 3, -8, 14, -10, 2, -11/2, 147/2, -45, 3, -16, 22, -18]

[2, 2, 8, 5, 4, 8, 6, 5, 26, 14, 10, 17, 12, 19/2, 14, 11, 46/5, 26, 18, 14, 20]

[0, 6, -3, -1, 4, -2, -1, 21, -12, -4, 7, -5, -5/2, 9/2, -3, -9/5, 84/5, -8, -4, 6]


In [37]:
data=[]
p=2
ds=1
mp=0
for pj in [0..20]:
    power=u[pj][0]
    if digitsum(p,power)==ds:
        poly=D(u[pj][1]) 
        for m in [3..30]:
            if mod(m,p)==mp:
                v=evaluate(poly,3) #coding error, but there is a hit for ud: A068156
                o=order(p,v)
                data=data+[-o]
print(data);print()
ud=uniteList(data)
print(ud)

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93]

[3, 9, 21, 45, 93]


In [68]:
data=[]
pairs=[]
p=2
ds=1
mp=0
no=[]
for pj in [0..len(u)-1]:
    k=u[pj][0]
    if digitsum(p,k)==ds:
        b=len(k.digits(p))-1 # so k = 2^b
        poly=D(u[pj][1]) 
        for m in [3..3]:
            v=evaluate(poly,3)  
            o=order(p,v)
            pairs=pairs+[(b,-o)]
            data=data+[-o]
            print((k,b,-o))
ud=uniteList(data)
print(ud) #A068156 
print()
up=uniteList(pairs);print(up)

(1, 0, 3)
(2, 1, 9)
(4, 2, 21)
(8, 3, 45)
(16, 4, 93)
(32, 5, 189)
(64, 6, 381)
[3, 9, 21, 45, 93, 189, 381]

[(0, 3), (1, 9), (2, 21), (3, 45), (4, 93), (5, 189), (6, 381)]


In [70]:
data=[]
pairs=[]
p=3
ds=1
mp=0
no=[]
for pj in [0..len(u)-1]:
    k=u[pj][0]
    if digitsum(p,k)==ds:
        b=len(k.digits(p))-1 # so power = 2^b
        poly=D(u[pj][1]) 
        for m in [3..3]:
            v=evaluate(poly,3)  
            o=order(p,v)
            pairs=pairs+[(b,-o)]
            data=data+[-o]
            print((k,b,-o))
ud=uniteList(data)
print(ud) 
print()
up=uniteList(pairs);print(up) # 3^n - 1

(1, 0, 2)
(3, 1, 8)
(9, 2, 26)
(27, 3, 80)
(81, 4, 242)
[2, 8, 26, 80, 242]

[(0, 2), (1, 8), (2, 26), (3, 80), (4, 242)]


In [71]:
data=[]
pairs=[]
p=5
ds=1
mp=0
no=[]
for pj in [0..len(u)-1]:
    k=u[pj][0]
    if digitsum(p,k)==ds:
        b=len(k.digits(p))-1 # so power = 2^b
        poly=D(u[pj][1]) 
        for m in [3..3]:
            v=evaluate(poly,m)  
            o=order(p,v)
            pairs=pairs+[(b,-o)]
            data=data+[-o]
            print((k,b,-o))
ud=uniteList(data)
print(ud) 
print()
up=uniteList(pairs);print(up)

(1, 0, 0)
(5, 1, 0)
(25, 2, 0)
[0]

[(0, 0), (1, 0), (2, 0)]


In [66]:
def mx(lst):
    if lst==[]:return -Infinity
    if lst!=[]:return(max(lst))
data=[]
pairs=[]
p=2
ds=1
mp=0
no=[]
m=6
for pj in [0..len(u)-1]:
    k=u[pj][0]
    if digitsum(p,k)==ds:
        b=len(k.digits(p))-1 # so k = 2^b
        poly=D(u[pj][1]) 
        v=evaluate(poly,m)  
        o=order(p,v)
        if (-o>mx(data))==False:no=no+[o]
        pairs=pairs+[(b,-o)]
        data=data+[-o]
        print((k,b,-o))
ud=uniteList(data) # 2^n-1
print(ud)
print()
up=uniteList(pairs);print(up)
print("decreases:",len(no))

(1, 0, 1)
(2, 1, 3)
(4, 2, 7)
(8, 3, 15)
(16, 4, 31)
(32, 5, 63)
(64, 6, 127)
[1, 3, 7, 15, 31, 63, 127]

[(0, 1), (1, 3), (2, 7), (3, 15), (4, 31), (5, 63), (6, 127)]
decreases: 0


In [72]:
def mx(lst):
    if lst==[]:return -Infinity
    if lst!=[]:return(max(lst))
data=[]
pairs=[]
p=3
ds=1
mp=0
no=[]
m=6
for pj in [0..len(u)-1]:
    k=u[pj][0]
    if digitsum(p,k)==ds:
        b=len(k.digits(p))-1 # so k = 2^b
        poly=D(u[pj][1]) 
        v=evaluate(poly,m)  
        o=order(p,v)
        if (-o>mx(data))==False:no=no+[o]
        pairs=pairs+[(b,-o)]
        data=data+[-o]
        print((k,b,-o))
ud=uniteList(data) # 3^n-1
print(ud) #A068156 
print()
up=uniteList(pairs);print(up)
print("decreases:",len(no))

(1, 0, 2)
(3, 1, 8)
(9, 2, 26)
(27, 3, 80)
(81, 4, 242)
[2, 8, 26, 80, 242]

[(0, 2), (1, 8), (2, 26), (3, 80), (4, 242)]
decreases: 0


In [73]:
def mx(lst):
    if lst==[]:return -Infinity
    if lst!=[]:return(max(lst))
data=[]
pairs=[]
p=5
ds=1
mp=0
no=[]
m=6
for pj in [0..len(u)-1]:
    k=u[pj][0]
    if digitsum(p,k)==ds:
        b=len(k.digits(p))-1 # so k = 2^b
        poly=D(u[pj][1]) 
        v=evaluate(poly,m)  
        o=order(p,v)
        if (-o>mx(data))==False:no=no+[o]
        pairs=pairs+[(b,-o)]
        data=data+[-o]
        print((k,b,-o))
ud=uniteList(data) # 3^n-1
print(ud) #A068156 
print()
up=uniteList(pairs);print(up)
print("decreases:",len(no))

(1, 0, 0)
(5, 1, 0)
(25, 2, 0)
[0]

[(0, 0), (1, 0), (2, 0)]
decreases: 2


In [74]:
def mx(lst):
    if lst==[]:return -Infinity
    if lst!=[]:return(max(lst))
data=[]
pairs=[]
p=4
ds=1
mp=0
no=[]
m=6
for pj in [0..len(u)-1]:
    k=u[pj][0]
    if digitsum(p,k)==ds:
        b=len(k.digits(p))-1 # so k = 2^b
        poly=D(u[pj][1]) 
        v=evaluate(poly,m)  
        o=order(p,v)
        if (-o>mx(data))==False:no=no+[o]
        pairs=pairs+[(b,-o)]
        data=data+[-o]
        print((k,b,-o))
ud=uniteList(data) # 4^n-1
print(ud) #A068156 
print()
up=uniteList(pairs);print(up)
print("decreases:",len(no))

(1, 0, 0)
(4, 1, 3)
(16, 2, 15)
(64, 3, 63)
[0, 3, 15, 63]

[(0, 0), (1, 3), (2, 15), (3, 63)]
decreases: 0


In [75]:
def mx(lst):
    if lst==[]:return -Infinity
    if lst!=[]:return(max(lst))
data=[]
pairs=[]
p=2
ds=1
mp=0
no=[]
m=4
for pj in [0..len(u)-1]:
    k=u[pj][0]
    if digitsum(p,k)==ds:
        b=len(k.digits(p))-1 # so k = 2^b
        poly=D(u[pj][1]) 
        v=evaluate(poly,m)  
        o=order(p,v)
        if (-o>mx(data))==False:no=no+[o]
        pairs=pairs+[(b,-o)]
        data=data+[-o]
        print((k,b,-o))
ud=uniteList(data) # 2^n-3
print(ud) #A068156 
print()
up=uniteList(pairs);print(up)
print("decreases:",len(no))

(1, 0, 5)
(2, 1, 13)
(4, 2, 29)
(8, 3, 61)
(16, 4, 125)
(32, 5, 253)
(64, 6, 509)
[5, 13, 29, 61, 125, 253, 509]

[(0, 5), (1, 13), (2, 29), (3, 61), (4, 125), (5, 253), (6, 509)]
decreases: 0


In [76]:
def mx(lst):
    if lst==[]:return -Infinity
    if lst!=[]:return(max(lst))
data=[]
pairs=[]
p=3
ds=1
mp=0
no=[]
m=4
for pj in [0..len(u)-1]:
    k=u[pj][0]
    if digitsum(p,k)==ds:
        b=len(k.digits(p))-1 # so k = 2^b
        poly=D(u[pj][1]) 
        v=evaluate(poly,m)  
        o=order(p,v)
        if (-o>mx(data))==False:no=no+[o]
        pairs=pairs+[(b,-o)]
        data=data+[-o]
        print((k,b,-o))
ud=uniteList(data) # 2^n-3
print(ud) #A068156 
print()
up=uniteList(pairs);print(up)
print("decreases:",len(no))

(1, 0, 0)
(3, 1, 0)
(9, 2, 0)
(27, 3, 0)
(81, 4, 0)
[0]

[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0)]
decreases: 4


In [77]:
def mx(lst):
    if lst==[]:return -Infinity
    if lst!=[]:return(max(lst))
data=[]
pairs=[]
p=3
ds=1
mp=0
no=[]
m=5
for pj in [0..len(u)-1]:
    k=u[pj][0]
    if digitsum(p,k)==ds:
        b=len(k.digits(p))-1 # so k = 2^b
        poly=D(u[pj][1]) 
        v=evaluate(poly,m)  
        o=order(p,v)
        if (-o>mx(data))==False:no=no+[o]
        pairs=pairs+[(b,-o)]
        data=data+[-o]
        print((k,b,-o))
ud=uniteList(data) # 2^n-3
print(ud) #A068156 
print()
up=uniteList(pairs);print(up)
print("decreases:",len(no))

(1, 0, 0)
(3, 1, 0)
(9, 2, 0)
(27, 3, 0)
(81, 4, 0)
[0]

[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0)]
decreases: 4


In [10]:
u[2][0]

3

In [58]:
# cn 3 analogue
data=[]
odata=[]
p=2
k=u[2][0] # = 3
poly=D(u[2][1]) # poly(m) = cst term of j_m^power
for a in [1..200]:
    m=p^a
    polym=evaluate(poly,m)
    o=order(p,polym)
    odata=odata+[o]
print(odata)

[-23, -18, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24]

In [57]:
# cn 3 analogue
data=[]
odata=[]
p=3
k=u[2][0] # = 3
poly=D(u[2][1]) # poly(m) = cst term of j_m^power
for a in [1..200]:
    m=p^a
    polym=evaluate(poly,m)
    o=order(p,polym)
    odata=odata+[o]
print(odata)

[-8, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14, -14]


In [59]:
# cn 3 analogue
data=[]
odata=[]
p=5
k=u[2][0] # = 3
poly=D(u[2][1]) # poly(m) = cst term of j_m^power
for a in [1..200]:
    m=p^a
    polym=evaluate(poly,m)
    o=order(p,polym)
    odata=odata+[o]
print(odata)

[-6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6]


In [60]:
# cn 3 analogue
data=[]
odata=[]
p=7
k=u[2][0] # = 3
poly=D(u[2][1]) # poly(m) = cst term of j_m^power
for a in [1..200]:
    m=p^a
    polym=evaluate(poly,m)
    o=order(p,polym)
    odata=odata+[o]
print(odata)

[-6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6]


In [61]:
# cn 3 analogue
data=[]
odata=[]
p=11
k=u[2][0] # = 3
poly=D(u[2][1]) # poly(m) = cst term of j_m^power
for a in [1..200]:
    m=p^a
    polym=evaluate(poly,m)
    o=order(p,polym)
    odata=odata+[o]
print(odata)

[-6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6, -6]


In [62]:
# cn 3 analogue
data=[]
odata=[]
p=13
k=u[2][0] # = 3
poly=D(u[2][1]) # poly(m) = cst term of j_m^power
for a in [1..200]:
    m=p^a
    polym=evaluate(poly,m)
    o=order(p,polym)
    odata=odata+[o]
print(odata)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [63]:
# cn 3 analogue
data=[]
odata=[]
p=2
k=u[2][0] # = 3
poly=D(u[2][1]) # poly(m) = cst term of j_m^power
for a in [1..200]:
    m=p^a+1
    polym=evaluate(poly,m)
    o=order(p,polym)
    odata=odata+[o]
print(odata)

[-12, -12, -12, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22, -22]

In [64]:
data=[]
odata=[]
p=2
k=u[2][0] # = 3
poly=D(u[2][1]) # poly(m) = cst term of j_m^power
for a in [1..200]:
    m=p^a+2
    polym=evaluate(poly,m)
    o=order(p,polym)
    odata=odata+[o]
print(odata)

[-18, -4, -4, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23]


In [65]:
data=[]
odata=[]
p=2
k=u[2][0] # = 3
poly=D(u[2][1]) # poly(m) = cst term of j_m^power
for a in [1..200]:
    m=p^a+3
    polym=evaluate(poly,m)
    o=order(p,polym)
    odata=odata+[o]
print(odata)

[-12, -12, -12, -19, -18, -17, -16, -15, -14, -13, -11, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12]

In [66]:
data=[]
odata=[]
p=2
k=u[2][0] # = 3
poly=D(u[2][1]) # poly(m) = cst term of j_m^power
for a in [1..200]:
    m=p^a+4
    polym=evaluate(poly,m)
    o=order(p,polym)
    odata=odata+[o]
print(odata)

[-4, -24, -18, -21, -20, -19, -17, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18]


In [67]:
data=[]
odata=[]
p=2
k=u[2][0] # = 3
poly=D(u[2][1]) # poly(m) = cst term of j_m^power
for a in [1..200]:
    m=p^a+5
    polym=evaluate(poly,m)
    o=order(p,polym)
    odata=odata+[o]
print(odata)

[-12, -12, -22, -21, -20, -19, -18, -17, -16, -15, -14, -13, -10, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12, -12]

In [68]:
data=[]
odata=[]
p=2
k=u[2][0] # = 3
poly=D(u[2][1]) # poly(m) = cst term of j_m^power
for a in [1..200]:
    m=p^a+6
    polym=evaluate(poly,m)
    o=order(p,polym)
    odata=odata+[o]
print(odata)

[-24, -4, -23, -22, -21, -20, -19, -18, -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -2, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4]


In [74]:
pairs=[]
odata=[]
lasts=[]
p=2
k=u[2][0] # = 3
poly=D(u[2][1]) # poly(m) = cst term of j_m^power
for b in [0..20]:
    for a in [1..200]:
        m=p^a+b
        polym=evaluate(poly,m)
        o=order(p,polym)
        odata=odata+[o]
    last=-odata[len(odata)-1]
    lasts=lasts+[last]
    pair=(b,last)
    pairs=pairs+[pair]
    print(pair)
print(pairs)
print(lasts)
print(sortAndSmash(lasts))

(0, 24)
(1, 22)
(2, 23)
(3, 12)
(4, 18)
(5, 12)
(6, 4)
(7, 12)
(8, 24)
(9, 12)
(10, 4)
(11, 12)
(12, 18)
(13, 22)
(14, 23)
(15, 21)
(16, 24)
(17, 22)
(18, 23)
(19, 19)
(20, 21)
[(0, 24), (1, 22), (2, 23), (3, 12), (4, 18), (5, 12), (6, 4), (7, 12), (8, 24), (9, 12), (10, 4), (11, 12), (12, 18), (13, 22), (14, 23), (15, 21), (16, 24), (17, 22), (18, 23), (19, 19), (20, 21)]
[24, 22, 23, 12, 18, 12, 4, 12, 24, 12, 4, 12, 18, 22, 23, 21, 24, 22, 23, 19, 21]
[4, 12, 18, 19, 21, 22, 23, 24]


In [76]:
data=[]
odata=[]
p=2
k=u[2][0] # = 3
poly=D(u[2][1]) # poly(m) = cst term of j_m^power
for a in [2..200]:
    m=p^a+p
    polym=evaluate(poly,m)
    o=order(p,polym)
    odata=odata+[o]
print(odata)

[-4, -4, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23, -23]


In [77]:
data=[]
odata=[]
p=2
k=u[2][0] # = 3
poly=D(u[2][1]) # poly(m) = cst term of j_m^power
for a in [3..200]:
    m=p^a+p^2
    polym=evaluate(poly,m)
    o=order(p,polym)
    odata=odata+[o]
print(odata)

[-18, -21, -20, -19, -17, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18, -18]


In [78]:
data=[]
odata=[]
p=2
k=u[2][0] # = 3
poly=D(u[2][1]) # poly(m) = cst term of j_m^power
for a in [4..200]:
    m=p^a+p^3
    polym=evaluate(poly,m)
    o=order(p,polym)
    odata=odata+[o]
print(odata)

[-24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24, -24]


In [79]:
data=[]
odata=[]
p=2
k=u[2][0] # = 3
poly=D(u[2][1]) # poly(m) = cst term of j_m^power
for m in [3..200]:
    if digitsum(p,m)==1:
        polym=evaluate(poly,m)
        o=order(p,polym)
        odata=odata+[o]
print(odata)

[-18, -24, -24, -24, -24, -24]


In [80]:
data=[]
odata=[]
p=2
k=u[2][0] # = 3
poly=D(u[2][1]) # poly(m) = cst term of j_m^power
for m in [3..200]:
    if digitsum(p,m)==2:
        polym=evaluate(poly,m)
        o=order(p,polym)
        odata=odata+[o]
print(odata)

[-12, -12, -4, -12, -4, -18, -22, -23, -21, -24, -22, -23, -20, -24, -24, -22, -23, -19, -24, -24, -24, -22, -23, -17, -24, -24, -24, -24]


In [11]:
odata=[]
tests=[]
for n in [2..50]:
    p=prime(n)
    lu=len(u)
    if (p>113)==False:
        k=u[p-1][0]
        poly=D(u[p-1][1]) 
        m=p
        polym=evaluate(poly,m)
        o=order(p,polym)
        odata=odata+[-o/4]
        test=(p+1)/2+o/4
        tests=tests+[test]
        print((p,k,test))
print(tests)


(3, 3, 0)
(5, 5, 0)
(7, 7, 0)
(11, 11, 0)
(13, 13, 0)
(17, 17, 0)
(19, 19, 0)
(23, 23, 0)
(29, 29, 0)
(31, 31, 0)
(37, 37, 0)
(41, 41, 0)
(43, 43, 0)
(47, 47, 0)
(53, 53, 0)
(59, 59, 0)
(61, 61, 0)
(67, 67, 0)
(71, 71, 0)
(73, 73, 0)
(79, 79, 0)
(83, 83, 0)
(89, 89, 0)
(97, 97, 0)
(101, 101, 0)
(103, 103, 0)
(107, 107, 0)
(109, 109, 0)
(113, 113, 0)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [6]:
odata=[]
tests=[]
for n in [2..50]:
    p=prime(n)
    lu=len(u)
    if (p>113)==False:
        k=u[p-1][0]
        poly=D(u[p-1][1]) 
        m=p
        polym=evaluate(poly,m)
        o=order(p,polym)
        odata=odata+[-o/4]
        test=o-(-2-2*p)
        tests=tests+[test]
        print((p,k,test))
print(tests)

(3, 3, 0)
(5, 5, 0)
(7, 7, 0)
(11, 11, 0)
(13, 13, 0)
(17, 17, 0)
(19, 19, 0)
(23, 23, 0)
(29, 29, 0)
(31, 31, 0)
(37, 37, 0)
(41, 41, 0)
(43, 43, 0)
(47, 47, 0)
(53, 53, 0)
(59, 59, 0)
(61, 61, 0)
(67, 67, 0)
(71, 71, 0)
(73, 73, 0)
(79, 79, 0)
(83, 83, 0)
(89, 89, 0)
(97, 97, 0)
(101, 101, 0)
(103, 103, 0)
(107, 107, 0)
(109, 109, 0)
(113, 113, 0)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [14]:
odata=[]
tests=[]
a=1
for n in [2..50]:
    p=prime(n)
    lu=len(u)
    if (p>113)==False:
        k=u[p-1][0]
        poly=D(u[p-1][1]) 
        m=p^a
        polym=evaluate(poly,m)
        o=order(p,polym)
        test=((-2-2*p)-o*k)/2
        tests=tests+[test]
        print((p,k,test))
print(tests)#A061066 after a transformation: (p^2-1)# so o = -2-2p
# bc k = p.

(3, 3, 8)
(5, 5, 24)
(7, 7, 48)
(11, 11, 120)
(13, 13, 168)
(17, 17, 288)
(19, 19, 360)
(23, 23, 528)
(29, 29, 840)
(31, 31, 960)
(37, 37, 1368)
(41, 41, 1680)
(43, 43, 1848)
(47, 47, 2208)
(53, 53, 2808)
(59, 59, 3480)
(61, 61, 3720)
(67, 67, 4488)
(71, 71, 5040)
(73, 73, 5328)
(79, 79, 6240)
(83, 83, 6888)
(89, 89, 7920)
(97, 97, 9408)
(101, 101, 10200)
(103, 103, 10608)
(107, 107, 11448)
(109, 109, 11880)
(113, 113, 12768)
[8, 24, 48, 120, 168, 288, 360, 528, 840, 960, 1368, 1680, 1848, 2208, 2808, 3480, 3720, 4488, 5040, 5328, 6240, 6888, 7920, 9408, 10200, 10608, 11448, 11880, 12768]


In [16]:
odata=[]
tests=[]
a=2
for n in [2..50]:
    p=prime(n)
    lu=len(u)
    if (p>113)==False:
        k=u[p-1][0]
        poly=D(u[p-1][1]) 
        m=p^a
        polym=evaluate(poly,m)
        o=order(p,polym)
        test=((-2-2*p)-o*k)
        tests=tests+[test]
        print((p,k,test))
print(tests)

(3, 3, 34)
(5, 5, 43)
(7, 7, 96)
(11, 11, 240)
(13, 13, 336)
(17, 17, 559)
(19, 19, 720)
(23, 23, 1056)
(29, 29, 1680)
(31, 31, 1920)
(37, 37, 2736)
(41, 41, 3360)
(43, 43, 3696)
(47, 47, 4416)
(53, 53, 5616)
(59, 59, 6960)
(61, 61, 7440)
(67, 67, 8976)
(71, 71, 10080)
(73, 73, 10656)
(79, 79, 12480)
(83, 83, 13776)
(89, 89, 15840)
(97, 97, 18816)
(101, 101, 20400)
(103, 103, 21216)
(107, 107, 22896)
(109, 109, 23760)
(113, 113, 25536)
[34, 43, 96, 240, 336, 559, 720, 1056, 1680, 1920, 2736, 3360, 3696, 4416, 5616, 6960, 7440, 8976, 10080, 10656, 12480, 13776, 15840, 18816, 20400, 21216, 22896, 23760, 25536]


In [17]:
lst=[34, 43, 96, 240, 336, 559, 720, 1056, 1680, 1920, 2736, 3360, 3696, 4416, 5616, 6960, 7440, 8976, 10080, 10656, 12480, 13776, 15840, 18816, 20400, 21216, 22896, 23760, 25536]
print(difference_table(lst))

[9, 53, 144, 96, 223, 161, 336, 624, 240, 816, 624, 336, 720, 1200, 1344, 480, 1536, 1104, 576, 1824, 1296, 2064, 2976, 1584, 816, 1680, 864, 1776]
